In [1]:
from transformations import Compose, Resize, DenseTarget
from transformations import MoveAxis, Normalize01
from customdatasets import SegmentationDataSet
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pathlib

# root directory
root = pathlib.Path.cwd() / 'Carvana'
def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
inputs = get_filenames_of_path(root / 'Input')
targets = get_filenames_of_path(root / 'Target')

# training transformations and augmentations
transforms = Compose([
    DenseTarget(),
    MoveAxis(),
    Normalize01()
])

# random seed
random_seed = 42

# split dataset into training set and validation set
train_size = 0.8  # 80:20 split

inputs_train, inputs_valid = train_test_split(
    inputs,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

targets_train, targets_valid = train_test_split(
    targets,
    random_state=random_seed,
    train_size=train_size,
    shuffle=True)

# dataset training
dataset_train = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms)

# dataset validation
dataset_valid = SegmentationDataSet(inputs=inputs_valid,
                                    targets=targets_valid,
                                    transform=transforms)

# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=2,
                                 shuffle=True)

# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=2,
                                   shuffle=True)


In [2]:
x, y = next(iter(dataloader_training))

print(f'x = shape: {x.shape}; type: {x.dtype}')
print(f'x = min: {x.min()}; max: {x.max()}')
print(f'y = shape: {y.shape}; class: {y.unique()}; type: {y.dtype}')

x = shape: torch.Size([2, 3, 1280, 1918]); type: torch.float32
x = min: 0.0; max: 1.0
y = shape: torch.Size([2, 1280, 1918]); class: tensor([0, 1]); type: torch.int64


In [3]:
%gui qt
from visual import Input_Target_Pair_Generator
from visual import show_input_target_pair_napari
gen = Input_Target_Pair_Generator(dataloader_training, rgb=True)
show_input_target_pair_napari(gen)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 639.5, 958.5), zoom=0.2859415753781951, angles=(0.0, 0.0, 90.0), interactive=True), cursor=Cursor(position=(0.0, 0.0), scaled=True, size=10, style='standard'), dims=Dims(ndim=2, ndisplay=2, last_used=1, range=((0.0, 1279.0, 1.0), (0.0, 1917.0, 1.0)), current_step=(0, 0), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'input_training' at 0x20a86a790a0>, <Labels layer 'target_training' at 0x20a86a79b50>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right'), active_layer=<Labels layer 'target_training' at 0x20a86a79b50>, help='enter paint or fill mode to edit labels', status='Ready', theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x0000020A80C60790>], _persisted_mouse_event={}, _mouse_drag_gen=

In [4]:
import albumentations
from transformations import Compose, AlbuSeg2d, DenseTarget, MoveAxis, Normalize01
# training transformations and augmentations
transforms_training = Compose([
    # Resize(input_size=(128, 128, 3), target_size=(128, 128)),
    AlbuSeg2d(albu=albumentations.HorizontalFlip(p=0.5)),
    DenseTarget(),
    MoveAxis(),
    Normalize01()
])

# validation transformations
transforms_validation = Compose([
    # Resize(input_size=(128, 128, 3), target_size=(128, 128)),
    DenseTarget(),
    MoveAxis(),
    Normalize01()
])

In [5]:
# dataset training
dataset_train = SegmentationDataSet(inputs=inputs_train,
                                    targets=targets_train,
                                    transform=transforms_training)

# dataset validation
dataset_valid = SegmentationDataSet(inputs=inputs_valid,
                                    targets=targets_valid,
                                    transform=transforms_validation)

# dataloader training
dataloader_training = DataLoader(dataset=dataset_train,
                                 batch_size=2,
                                 shuffle=True)

# dataloader validation
dataloader_validation = DataLoader(dataset=dataset_valid,
                                   batch_size=2,
                                   shuffle=True)

In [6]:
%gui qt
from visual import Input_Target_Pair_Generator
from visual import show_input_target_pair_napari
gen_train = Input_Target_Pair_Generator(dataloader_training, rgb=True)
gen_valid = Input_Target_Pair_Generator(dataloader_validation, rgb=True)
show_input_target_pair_napari(gen_train, gen_valid)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 639.5, 958.5), zoom=0.2859415753781951, angles=(0.0, 0.0, 90.0), interactive=True), cursor=Cursor(position=(0.0, 0.0), scaled=True, size=10, style='standard'), dims=Dims(ndim=2, ndisplay=2, last_used=1, range=((0.0, 1279.0, 1.0), (0.0, 1917.0, 1.0)), current_step=(0, 0), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'input_training' at 0x20a97119760>, <Labels layer 'target_training' at 0x20a97060be0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right'), active_layer=<Labels layer 'target_training' at 0x20a97060be0>, help='enter paint or fill mode to edit labels', status='Ready', theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x0000020A80C60790>], _persisted_mouse_event={}, _mouse_drag_gen=